In [1]:
import spacy
import torchtext
import pandas as pd

import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim
from torch.autograd import Variable

import warnings

warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('../data/training_data.csv')
df_train.sample(5)

,Intent,Sentence
1974,1,Hi: please add $4 of tolls for each day that I...
1146,0,"In styles for every decor, at prices to please..."
2790,1,"Please set the meeting up with Alan, Dan and me."
901,0,I understand that there may be a need to give ...
2382,1,Do you run a business where you need to send p...


In [3]:
spacy_en      = spacy.load('en')
embedding_dim = 100
fix_length    = 128
epochs        = 20
print_every   = 30

In [4]:
def en_tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
TEXT = torchtext.data.Field(
    sequential = True,
    tokenize   = en_tokenizer,
    lower      = True,
    fix_length = fix_length # for network needing fixed length inputs 
)

In [6]:
LABEL = torchtext.data.Field(
    sequential = False,
    use_vocab  = False
)

In [7]:
train_data, test_data = torchtext.data.TabularDataset.splits(
    path        = '../data/',
    train       = 'training_data.csv',
    test        = 'test_data.csv',
    format      = 'csv',
    skip_header = True,
    fields      = [
        ('Intent'  , LABEL),
        ('Sentence', TEXT)
    ]
)

In [8]:
train_iter, test_iter = torchtext.data.Iterator.splits(
    (train_data, test_data),
    sort_key    = lambda x: len(x.Sentence),
    batch_sizes = (32, 256),
    device      = -1, # -1 for CPU, 0 for GPU
    repeat      = False
)

In [9]:
def init_unk_emb(vocab, init = 'randn', num_special_toks = 2):
    emb_vectors = vocab.vectors
    sweep_range = len(vocab)
    running_norm = 0.
    num_non_zero = 0
    total_words = 0
    
    for i in range(num_special_toks, sweep_range):
        if len(emb_vectors[i, :].nonzero()) == 0:
            if init == 'randn':
                torch.nn.init.normal(emb_vectors[i], mean = 0, std = 0.05)
        else:
            num_non_zero += 1
            running_norm += torch.norm(emb_vectors[i])
        total_words += 1
    print(f'average GloVE norm {running_norm / num_non_zero}, '
          f'known words {num_non_zero}, '
          f'total words {total_words}')

In [10]:
embedding_fn = f'glove.6B.{embedding_dim}d'
TEXT.build_vocab(train_data, vectors = embedding_fn)
vocab = TEXT.vocab
init_unk_emb(vocab)

average GloVE norm 5.217196088951167, known words 6145, total words 6874


In [11]:
def evaluate_model(model, criterion, train_iter, test_iter):
    def compute_loss_and_accuracy(dataset):
        batch_number      = len(dataset)
        total_loss        = 0
        total_predictions = 0
        total_correct     = 0
        for batch_data in dataset:
            X, y               = batch_data.Sentence, batch_data.Intent
            y_pred             = model(X)
            loss               = criterion(y_pred, y)
            total_loss        += loss.data[0]
            correct_guesses    = (y_pred.max(dim = 1)[1] == y).sum().data[0]
            total_predictions += len(y)
            total_correct     += correct_guesses
            
        accuracy     = total_correct / total_predictions
        average_loss = total_loss / batch_number

        return accuracy, average_loss
    
    model.eval()
    eval_train = compute_loss_and_accuracy(train_iter)
    eval_test  = compute_loss_and_accuracy(test_iter)
    model.train()
    
    return eval_train, eval_test

In [12]:
class CNN(nn.Module):
    def __init__(self, vocab):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embedding_dim)
        self.conv1     = nn.Conv1d(embedding_dim, 32, 3, padding = 1)
        self.conv2     = nn.Conv1d(32, 32, 3, padding = 1)
        self.conv3     = nn.Conv1d(32, 64, 3, padding = 1)
        self.conv4     = nn.Conv1d(64, 64, 3, padding = 1)
        self.linear1   = nn.Linear(64 * 32, 2)
        
        self.embedding.weight.data.copy_(vocab.vectors)
        
    def forward(self, x):
        x = x.transpose(0, 1)
        x = self.embedding(x)
        x = x.transpose(1, 2)
        x = F.dropout(F.relu(self.conv1(x)), p = 0.7)
        x = F.dropout(F.relu(self.conv2(x)), p = 0.7)
        x = F.max_pool1d(x, 2)
        x = F.dropout(F.relu(self.conv3(x)), p = 0.5)
        x = F.dropout(F.relu(self.conv4(x)), p = 0.5)
        x = F.max_pool1d(x, 2)
        x = x.view(-1, 64 * 32)
        x = self.linear1(x)
        
        return x

In [13]:
model     = CNN(vocab)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [14]:
for epoch in range(epochs):
    running_loss = 0
    for i, batch_data in enumerate(train_iter):
        optimizer.zero_grad()
        X, y          = batch_data.Sentence, batch_data.Intent
        y_pred        = model(X)
        loss          = criterion(y_pred, y)
        running_loss += loss.data[0]
        loss.backward()
        optimizer.step()
        
        
        if i % print_every == print_every - 1:
            print(f'\t[{i + 1:5}] Running loss: {running_loss / print_every : .5f}')
            running_loss = 0

    (train_acc, train_loss), (test_acc, test_loss) = evaluate_model(
        model,
        criterion, 
        train_iter, 
        test_iter
    )
    print(f'Epoch {epoch}: train acc {train_acc * 100 : .3f}% '
          f'train loss {train_loss : .5f} '
          f'test acc {test_acc * 100 : .3f}% '
          f'test loss {test_loss : .5f}')

	[   30] Running loss:  0.68909
	[   60] Running loss:  0.66972
	[   90] Running loss:  0.62652
Epoch 0: train acc  73.229% train loss  0.54892 test acc  82.258% test loss  0.48735
	[   30] Running loss:  0.50563
	[   60] Running loss:  0.56468
	[   90] Running loss:  0.47055
Epoch 1: train acc  83.538% train loss  0.39386 test acc  79.133% test loss  0.45007
	[   30] Running loss:  0.37075
	[   60] Running loss:  0.41177
	[   90] Running loss:  0.37560
Epoch 2: train acc  88.050% train loss  0.29005 test acc  80.040% test loss  0.44299
	[   30] Running loss:  0.31796
	[   60] Running loss:  0.29149
	[   90] Running loss:  0.30715
Epoch 3: train acc  92.808% train loss  0.19000 test acc  81.048% test loss  0.44214
	[   30] Running loss:  0.19484
	[   60] Running loss:  0.25139
	[   90] Running loss:  0.21588
Epoch 4: train acc  93.109% train loss  0.18132 test acc  72.782% test loss  0.64453
	[   30] Running loss:  0.12600
	[   60] Running loss:  0.13545
	[   90] Running loss:  0.13962